In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import os
import gensim
import time
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

In [2]:
CONSTANTS = {
    'label': 'default payment next month',
    'path': 'preprocessed_upsampled.csv',
    'sequence_features': ['PAY_', 'BILL_AMT', 'PAY_AMT'],
    'non_sequence_features': ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE'],
    'length': 6,
    'batch_size': 64,
}

In [3]:
class Dataset_seq(Dataset):
  def __init__(self, path):
    self.data = pd.read_csv(path)
    self.label = CONSTANTS['label']
    self.features = list(self.data.columns)
    self.features.remove(self.label)
  
  def __getitem__(self, index):
    ex = self.data.iloc[index]
    label = ex[self.label]
    features = ex[self.features]

    non_sequential_features = list(features[CONSTANTS['non_sequence_features']].values)

    all_features = []

    for i in range(1, 1 + CONSTANTS['length']):
      seq_i = []
      for base_feature in CONSTANTS['sequence_features']:
        seq_i.append(features[f'{base_feature}{i}'])
      
      seq_i += non_sequential_features
      all_features.append(seq_i)
    
    all_features = np.array(all_features)
    all_features = all_features.astype(np.double)
    all_features = torch.from_numpy(all_features)

    all_features = all_features.type(torch.float)

    return torch.flatten(all_features), torch.tensor(label, dtype=torch.long)
  
  def __len__(self):
    return self.data.shape[0]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = Dataset_seq(CONSTANTS['path'])

# Two Layer GRU

In [24]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
# Define RNN

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.hidden = 150
        self.layers = 2
        self.lstm = nn.GRU(input_size=48,hidden_size=self.hidden,num_layers=self.layers)
        self.linear = nn.Linear(self.hidden,2)
        self.relu = nn.ReLU()

    def forward(self, x, h):
        out, h = self.lstm(x,h)
        
        out = self.linear(out)
        return out,h

    def init_h(self, bsz):
        weight = next(self.parameters())
        return weight.new_zeros(self.layers, self.hidden)

    # def init_c(self, bsz):
    #     weight = next(self.parameters())
    #     return weight.new_zeros(self.layers, self.hidden)

Using cuda device


In [25]:
# Define Loss, Optimizer
model2 = NeuralNetwork()
model2 = model2.to(device)
lr = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',patience=4,threshold=1e-3, factor=0.4)



In [26]:
# Training 
batch_sz = CONSTANTS['batch_size']
data_size = len(data)
train_set, valid_set = random_split(data, [40000, 6728])

train_loader = DataLoader(train_set, batch_size=batch_sz, shuffle=True)
test_loader = DataLoader(valid_set, batch_size=batch_sz, shuffle=True)
# total_loader = DataLoader(data,batch_size = batch_sz, shuffle=True)

In [27]:
def metrics(y_true:torch.Tensor, y_pred:torch.Tensor, is_training=False) -> torch.Tensor:
    '''Calculate F1 score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. 0 <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    
    '''
    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)
        
    
    tp = (y_true * y_pred).sum().to(torch.float32)
    tn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)
    
    epsilon = 1e-7
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = 2* (precision*recall) / (precision + recall + epsilon)
    f1.requires_grad = is_training
    
    return torch.round(precision, decimals=3), torch.round(recall, decimals=3), torch.round(f1, decimals=3)

In [28]:
# train_loader = total_loader

def train2():
  
  h = model2.init_h(batch_sz)
  # c = model2.init_c(batch_sz)

  epochs = 25

  training_loss = []
  training_error = []
  training_precision = []
  training_recall = []
  training_f1 = []
  val_loss = []
  val_error = []
  val_precision = []
  val_recall = []
  val_f1 = []
  learning_rates = []
  for epoch in range(epochs):
    model2.train()
    start_time = time.time()
    running_loss = 0
    correct = 0
    flag = 0
    preds, targs = [], []

    for op_params in optimizer.param_groups:
      learning_rates.append(op_params['lr'])

    for input,label in train_loader:

        input = input.to(device)
        label = label.to(device)
        model2.zero_grad()
        h = h.detach()
        # c = c.detach()
        output, h = model2(input, h)
        output = output.to(device)

        loss = criterion(output, label)

        _, pred = torch.max(output, 1)
        
        correct += (pred == label).float().sum().detach().cpu()
        loss.backward()

        preds.extend(list(pred.detach().cpu()))
        targs.extend(label.tolist())

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        # torch.nn.utils.clip_grad_norm_(model2.parameters(), 0.2)
        optimizer.step()

        running_loss += loss.item()

        

    precision, recall, f1 = metrics(torch.tensor(preds), torch.tensor(targs))
    training_loss.append(running_loss/(len(train_loader)))
    training_error.append(1 - (correct/(len(train_loader)*batch_sz)))
    training_precision.append(precision)
    training_recall.append(recall)
    training_f1.append(f1)

    print('Epoch [{}/{}], Training Loss: {:.4f}, Training Error: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1: {:.4f}'.format(epoch+1, epochs, running_loss/(len(train_loader)), 1 - (correct/(len(train_loader)*batch_sz)), precision, recall, f1))


    



  # # Calculate validation loss and error after each epoch

    model2.eval()
    val_correct = 0
    running_loss_val = 0

    val_preds, val_targs = [], []
   

    for input,label in test_loader:
      input = input.to(device)
      label = label.to(device)

      output, _ = model2(input, h)

      loss = criterion(output, label)
      _, pred = torch.max(output, 1)
      val_correct += (pred == label).float().sum().detach().cpu()

      running_loss_val += loss.item()

      val_preds.extend(list(pred.detach().cpu()))
      val_targs.extend(label.tolist())

    precision, recall, f1 = metrics(torch.tensor(val_preds), torch.tensor(val_targs))
    avg_loss = running_loss_val/len(test_loader)
    val_loss.append(avg_loss)
    val_error.append(1 - (val_correct/(len(test_loader)*batch_sz)))
    val_precision.append(precision)
    val_recall.append(recall)
    val_f1.append(f1)

    print('Epoch [{}/{}], Validation Loss: {:.4f}, Validation Error: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1: {:.4f}'.format(epoch+1, epochs, avg_loss, 1 - (val_correct/(len(test_loader)*batch_sz)), precision, recall, f1))
    print("="*30)

    scheduler.step(avg_loss)

  return training_error, training_loss, training_precision, training_recall, training_f1, val_error, val_loss, val_precision, val_recall, val_f1, learning_rates



In [ ]:
training_error, training_loss, training_precision, training_recall, training_f1, val_error, val_loss, val_precision, val_recall, val_f1, learning_rates = train2()

In [ ]:
plt.plot(training_error)

In [ ]:
plt.plot(training_loss)

In [ ]:
plt.plot(val_error)

In [ ]:
plt.plot(val_loss)

In [ ]:
plt.plot(learning_rates)